# Storm Events Database

The purpose of this notebook is to download, assemble, and work with the storm events dataset.  This notebook will have the following steps:

1. Connect to the FTP server. "ftp.ncei.noaa.gov"
2. Download zipped CSV files of type "details", "fatalities", and "locations".
3. Decompress and load each of these files.
4. Join them using the 'Event_ID' column.
5. Delete individual csv and gz files
6. Save the csv file
7. Work with the data

The database currently contains data from January 1950 to April 2024 and is updated monthly.  Data on all event types starts in 1996, so we will only download files from 1996 and after.  Full documentation can be found here: https://www.ncdc.noaa.gov/stormevents/details.jsp

While there is some overlap with SHELDUS, the storm events database offers more detailed information at finer geographic scales.

## Needed Imports

In [9]:
from ftplib import FTP
import os
import pandas as pd
import gzip
import time
import glob
import requests
from bs4 import BeautifulSoup


## Connect to the FTP Server and get a list of files

In [2]:
ftp_server = "ftp.ncei.noaa.gov"
ftp_dir = "/pub/data/swdi/stormevents/csvfiles/"
ftp_conn = FTP(ftp_server)
ftp_conn.login()

files = ftp_conn.nlst(ftp_dir)
csv_files = [file for file in files if file.endswith('.csv.gz')]

In [4]:
sorted(csv_files)

['/pub/data/swdi/stormevents/csvfiles/StormEvents_details-ftp_v1.0_d1950_c20210803.csv.gz',
 '/pub/data/swdi/stormevents/csvfiles/StormEvents_details-ftp_v1.0_d1951_c20210803.csv.gz',
 '/pub/data/swdi/stormevents/csvfiles/StormEvents_details-ftp_v1.0_d1952_c20210803.csv.gz',
 '/pub/data/swdi/stormevents/csvfiles/StormEvents_details-ftp_v1.0_d1953_c20210803.csv.gz',
 '/pub/data/swdi/stormevents/csvfiles/StormEvents_details-ftp_v1.0_d1954_c20210803.csv.gz',
 '/pub/data/swdi/stormevents/csvfiles/StormEvents_details-ftp_v1.0_d1955_c20210803.csv.gz',
 '/pub/data/swdi/stormevents/csvfiles/StormEvents_details-ftp_v1.0_d1956_c20210803.csv.gz',
 '/pub/data/swdi/stormevents/csvfiles/StormEvents_details-ftp_v1.0_d1957_c20210803.csv.gz',
 '/pub/data/swdi/stormevents/csvfiles/StormEvents_details-ftp_v1.0_d1958_c20210803.csv.gz',
 '/pub/data/swdi/stormevents/csvfiles/StormEvents_details-ftp_v1.0_d1959_c20210803.csv.gz',
 '/pub/data/swdi/stormevents/csvfiles/StormEvents_details-ftp_v1.0_d1960_c202108

In [6]:
page = requests.get("https://www.ncei.noaa.gov/pub/data/swdi/stormevents/csvfiles/").text

In [13]:
url = "https://www.ncei.noaa.gov/pub/data/swdi/stormevents/csvfiles/"
ext = "csv.gz"
soup = BeautifulSoup(page, "html.parser")
csv_files_https = [url  + node.get('href') for node in soup.find_all('a') if node.get('href').endswith(ext)]

In [16]:
pd.read_csv(csv_files_https[50])

,BEGIN_YEARMONTH,BEGIN_DAY,BEGIN_TIME,END_YEARMONTH,END_DAY,END_TIME,EPISODE_ID,EVENT_ID,STATE,STATE_FIPS,...,END_RANGE,END_AZIMUTH,END_LOCATION,BEGIN_LAT,BEGIN_LON,END_LAT,END_LON,EPISODE_NARRATIVE,EVENT_NARRATIVE,DATA_SOURCE
0,200012,31,600,200012,31,900,1104812,5165377,FLORIDA,12,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Temperatures fell into the mid-20s over Glades...,NaN,PDC
1,200012,31,600,200012,31,900,1104812,5165378,FLORIDA,12,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Temperatures fell into the mid-20s over Glades...,NaN,PDC
2,200012,31,700,200012,31,800,1104812,5165379,FLORIDA,12,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Temperatures fell into the mid-20s over Glades...,NaN,PDC
3,200012,13,2200,200012,14,400,1105342,5165449,WEST VIRGINIA,54,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"A mix of sleet, freezing rain and snow spread ...",NaN,PDC
4,200008,3,1410,200008,3,1410,1101140,5172568,MISSISSIPPI,28,...,NaN,NaN,FORKVILLE,32.45,-89.65,32.45,-89.65,NaN,Several trees were blown down along and onto h...,PDC
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52002,200001,11,500,200001,11,900,2414768,5126692,WEST VIRGINIA,54,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,A secondary cold front caused early morning wi...,NaN,PDC
52003,200001,25,1030,200001,25,1030,2414731,5127563,NEW JERSEY,34,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,A strong low pressure system (Nor'easter) move...,NaN,PDC
52004,200002,1,0,200002,29,2359,2414804,5127165,GEORGIA,13,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Rainfall amounts for the month of February we...,NaN,PDC
52005,200002,1,0,200002,29,2359,2414804,5127830,GEORGIA,13,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Rainfall amounts for the month of February we...,NaN,PDC


## Filter files by types and download

We are going to download all zipped files from 1996 to the present.

In [3]:
#set the directory
data_dir = "./data"
os.makedirs(data_dir, exist_ok=True) 

In [4]:
# Filtering and downloading
details_files = [file for file in csv_files if "details" in file]
fatalities_files = [file for file in csv_files if "fatalities" in file]
locations_files = [file for file in csv_files if "locations" in file]

In [5]:
def download_and_decompress(files, prefix, timeout=5):
    dataframes = []
    for file in files:
        # Extract the year from the file name
        filename = os.path.basename(file)
        year_str = filename.split('_')[3][1:5]
        
        try:
            year = int(year_str)
        except ValueError:
            continue  # If the year is not a valid number, skip this file

        # Process files from 1996 and higher
        if year >= 1996:
            local_file_gz = os.path.join(data_dir, filename)
            
            # Download the file
            with open(local_file_gz, 'wb') as f:
                ftp_conn.retrbinary("RETR " + file, f.write)

            # Decompress and read the file
            with gzip.open(local_file_gz, 'rt') as f:
                df = pd.read_csv(f, dtype=str)
                dataframes.append(df)

            # Timeout between processing files
            time.sleep(timeout)

    # Combine all dataframes and save to a CSV file
    if dataframes:
        combined_df = pd.concat(dataframes, ignore_index=True)
        combined_df.to_csv(os.path.join(data_dir, f"{prefix}.csv"), index=False)

# Call the function for each set of files with a timeout of 5 seconds
download_and_decompress(details_files, "details", timeout=5)
download_and_decompress(fatalities_files, "fatalities", timeout=5)
download_and_decompress(locations_files, "locations", timeout=5)

# Close the FTP connection
ftp_conn.quit()

'221 Goodbye.'

### Load the CSVs into pandas and join using EVENT_ID

In [6]:
# Step 4: Load CSVs into pandas and join using 'Event_ID'
df_details = pd.read_csv(os.path.join(data_dir, "details.csv"), dtype=str)
df_fatalities = pd.read_csv(os.path.join(data_dir, "fatalities.csv"), dtype=str)
df_locations = pd.read_csv(os.path.join(data_dir, "locations.csv"), dtype=str)

merged_df = df_details.merge(df_fatalities, on="EVENT_ID", how="left")\
                      .merge(df_locations, on="EVENT_ID", how="left")

In [7]:
merged_df.head(5)

,BEGIN_YEARMONTH,BEGIN_DAY,BEGIN_TIME,END_YEARMONTH,END_DAY,END_TIME,EPISODE_ID_x,EVENT_ID,STATE,STATE_FIPS,...,YEARMONTH,EPISODE_ID_y,LOCATION_INDEX,RANGE,AZIMUTH,LOCATION,LATITUDE,LONGITUDE,LAT2,LON2
0,200604,7,1515,200604,7,1515,1207534,5501658,INDIANA,18,...,200604,1207534,1,4,E,PATOKA,38.41667,-87.51667,NaN,NaN
1,200604,7,1515,200604,7,1515,1207534,5501658,INDIANA,18,...,200604,1207534,2,4,E,PATOKA,38.33333,-87.35,NaN,NaN
2,200601,1,0,200601,31,2359,1202408,5482463,COLORADO,8,...,200601,1202408,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,200601,1,0,200601,31,2359,1202408,5482464,COLORADO,8,...,200601,1202408,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,200601,1,0,200601,31,2359,1202408,5482465,COLORADO,8,...,200601,1202408,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Deleting individual files

Let's clean up the downloaded files since we don't need them anymore and they are just taking up space

In [8]:
# List of individual files to delete
individual_files = ["details.csv", "fatalities.csv", "locations.csv"]
gz_files = [file.replace('.csv', '.csv.gz') for file in individual_files]

# Combine both lists
files_to_delete = individual_files + gz_files

# Delete specified files
for file in files_to_delete:
    file_path = os.path.join(data_dir, file)
    if os.path.exists(file_path):
        os.remove(file_path)
        print(f"Deleted: {file_path}")
    else:
        print(f"{file_path} not found.")

# Additionally, ensure all .gz files in the directory are deleted
gz_files_in_dir = glob.glob(os.path.join(data_dir, "*.gz"))
for gz_file in gz_files_in_dir:
    os.remove(gz_file)
    print(f"Deleted: {gz_file}")

Deleted: ./data\details.csv
Deleted: ./data\fatalities.csv
Deleted: ./data\locations.csv
./data\details.csv.gz not found.
./data\fatalities.csv.gz not found.
./data\locations.csv.gz not found.
Deleted: ./data\StormEvents_details-ftp_v1.0_d1996_c20220425.csv.gz
Deleted: ./data\StormEvents_details-ftp_v1.0_d1997_c20220425.csv.gz
Deleted: ./data\StormEvents_details-ftp_v1.0_d1998_c20220425.csv.gz
Deleted: ./data\StormEvents_details-ftp_v1.0_d1999_c20220425.csv.gz
Deleted: ./data\StormEvents_details-ftp_v1.0_d2000_c20220425.csv.gz
Deleted: ./data\StormEvents_details-ftp_v1.0_d2001_c20220425.csv.gz
Deleted: ./data\StormEvents_details-ftp_v1.0_d2002_c20220425.csv.gz
Deleted: ./data\StormEvents_details-ftp_v1.0_d2003_c20220425.csv.gz
Deleted: ./data\StormEvents_details-ftp_v1.0_d2004_c20220425.csv.gz
Deleted: ./data\StormEvents_details-ftp_v1.0_d2005_c20220425.csv.gz
Deleted: ./data\StormEvents_details-ftp_v1.0_d2006_c20220425.csv.gz
Deleted: ./data\StormEvents_details-ftp_v1.0_d2007_c2024021

## Working with the dataset

Let's focus on events in the area where the survey was administered (Southern California) and look at events since the start of the year. For this we'll be using County FIPs codes for the counties where the survey is administered.

### Using FIPS codes
The Storm Events Database has a column (CZ_FIPS) which uses the last three numbers of the FIPS code.  We'll do a boolean filter using that and the state code (STATE_FIPS), which is 06, but the dataset drops the leading zero.

In [9]:
socal_fips_codes = ['083', '111', '037', '031', '073', '043', '053', '027', '071', '069', '065', '029', '019', '039', '047', '107', '059', '025', '079']

In [10]:
# Define the start and end dates as strings.  The format is XXXXMM (Year/Month)
start_date = '202401'
end_date = '202404'

In [11]:
# Filter the DataFrame for rows between the start and end dates
filtered_df = merged_df[(merged_df['BEGIN_YEARMONTH'] >= start_date) & (merged_df['END_YEARMONTH'] <= end_date)]

In [12]:
# Filter the DataFrame based on the list of FIPS codes
fips = filtered_df[(filtered_df['CZ_FIPS'].isin(socal_fips_codes)) & (filtered_df['STATE_FIPS'] == '6')]
fips.head()

,BEGIN_YEARMONTH,BEGIN_DAY,BEGIN_TIME,END_YEARMONTH,END_DAY,END_TIME,EPISODE_ID_x,EVENT_ID,STATE,STATE_FIPS,...,YEARMONTH,EPISODE_ID_y,LOCATION_INDEX,RANGE,AZIMUTH,LOCATION,LATITUDE,LONGITUDE,LAT2,LON2
2064107,202401,3,606,202401,3,806,186834,1146696,CALIFORNIA,6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2064707,202403,2,1738,202403,2,1738,188027,1154871,CALIFORNIA,6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2065232,202402,6,430,202402,6,430,187518,1151475,CALIFORNIA,6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2065375,202402,19,1315,202402,19,1515,187711,1152775,CALIFORNIA,6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2065376,202402,19,1319,202402,19,1519,187711,1152776,CALIFORNIA,6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
# What kinds of events occurred?  Group by the 'EVENT_TYPE' column
event_counts = fips.groupby('EVENT_TYPE').size().reset_index(name='Count')
event_counts

,EVENT_TYPE,Count
0,Flash Flood,16
1,Flood,9
2,Funnel Cloud,5
3,Hail,1
4,Thunderstorm Wind,3


At this point, you can drill down to determine if the events in the dataset are relevant.

In [14]:
# Subset the DataFrame based on flash floods
fips_flash_flood = fips[fips['EVENT_TYPE'] == 'Flash Flood']
fips_flash_flood

,BEGIN_YEARMONTH,BEGIN_DAY,BEGIN_TIME,END_YEARMONTH,END_DAY,END_TIME,EPISODE_ID_x,EVENT_ID,STATE,STATE_FIPS,...,YEARMONTH,EPISODE_ID_y,LOCATION_INDEX,RANGE,AZIMUTH,LOCATION,LATITUDE,LONGITUDE,LAT2,LON2
2079395,202402,4,1956,202402,4,2115,189405,1164907,CALIFORNIA,6,...,202402,189405,1,1.71,W,VENTURA,34.2815,-119.2999,3416890,11917994
2079396,202402,4,1956,202402,4,2115,189405,1164907,CALIFORNIA,6,...,202402,189405,2,1.66,W,VENTURA,34.278,-119.2991,3416680,11917946
2079397,202402,4,1956,202402,4,2115,189405,1164907,CALIFORNIA,6,...,202402,189405,3,1.54,W,VENTURA,34.278,-119.2969,3416680,11917814
2079398,202402,4,1956,202402,4,2115,189405,1164907,CALIFORNIA,6,...,202402,189405,4,1.57,W,VENTURA,34.2816,-119.2974,3416896,11917844
2079630,202402,5,243,202402,5,430,189405,1164909,CALIFORNIA,6,...,202402,189405,1,10.45,SE,SANTA SUSANA,34.1802,-118.5529,3410812,11833174
2079631,202402,5,243,202402,5,430,189405,1164909,CALIFORNIA,6,...,202402,189405,2,10.43,SE,SANTA SUSANA,34.181,-118.5525,3410860,11833150
2079632,202402,5,243,202402,5,430,189405,1164909,CALIFORNIA,6,...,202402,189405,3,10.71,SE,SANTA SUSANA,34.1806,-118.5468,3410836,11832808
2079633,202402,5,243,202402,5,430,189405,1164909,CALIFORNIA,6,...,202402,189405,4,10.73,SE,SANTA SUSANA,34.18,-118.5469,3410800,11832814
2079875,202402,4,1735,202402,4,1930,189405,1164922,CALIFORNIA,6,...,202402,189405,1,1.5,NNE,EL RIO,34.25,-119.16,3415000,1199600
2079876,202402,4,1735,202402,4,1930,189405,1164922,CALIFORNIA,6,...,202402,189405,2,1.5,NNE,EL RIO,34.2497,-119.1588,3414982,1199528


In [15]:
# Let's look at injuries, deaths, property damage, and narrative related to these flash floods.
print(fips_flash_flood[['EVENT_TYPE', 'LOCATION', 'INJURIES_DIRECT', 'INJURIES_INDIRECT', 'DEATHS_DIRECT', 'DEATHS_INDIRECT', 'EVENT_NARRATIVE', 'DAMAGE_PROPERTY']])  

          EVENT_TYPE      LOCATION INJURIES_DIRECT INJURIES_INDIRECT  \
2079395  Flash Flood       VENTURA               0                 0   
2079396  Flash Flood       VENTURA               0                 0   
2079397  Flash Flood       VENTURA               0                 0   
2079398  Flash Flood       VENTURA               0                 0   
2079630  Flash Flood  SANTA SUSANA               0                 0   
2079631  Flash Flood  SANTA SUSANA               0                 0   
2079632  Flash Flood  SANTA SUSANA               0                 0   
2079633  Flash Flood  SANTA SUSANA               0                 0   
2079875  Flash Flood        EL RIO               0                 0   
2079876  Flash Flood        EL RIO               0                 0   
2079877  Flash Flood       SATICOY               0                 0   
2079878  Flash Flood       SATICOY               0                 0   
2082264  Flash Flood     BARDSDALE               0              

## Saving the dataset

In case we to use it for later.

In [16]:
#In case we want to save this dataset for later use, let's go ahead an save it.

file_name='stormevents_1996_2024.csv'
merged_df.to_csv(file_name)

In [ ]:
# Define the content of the README.md file
readme_content = """
# Working with the Storm Events D
A brief description of what this project does and who it's for.

## Table of Contents

- [Installation](#installation)
- [Usage](#usage)
- [Contributing](#contributing)
- [License](#license)

## Installation

Instructions on how to install the project or any dependencies.

## Usage

Examples of how to use the project. Include code snippets or screenshots as needed.

## Contributing

Guidelines for contributing to the project.

## License

Include licensing information here.
"""

# Write the content to README.md
with open('README.md', 'w') as f:
    f.write(readme_content)

# Confirm creation and display the content of the README.md
print("README.md created with the following content:")
print(readme_content)